# Fisher Class

In [1]:
import numpy as np
import sympy as smp
import matplotlib.pyplot as plt
import pandas as pd

# For interpolation
from scipy.interpolate import RectBivariateSpline, interp2d

import warnings

# Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*divmax.*") 

import logging
# Basic registry settings
logging.basicConfig(level=logging.INFO)

In [2]:
import Cosmo_util_data as cu
import Cosmo_integration as ci
import Cosmo_shear as cs

Interpolation of matter power spectrum created.
Interpolation of derivative of matter power spectrum with respect to k created.
Interpolation of luminosity function created.


In [3]:
# Parametros fiduciales

Omega_b0_fid = 0.05
Omega_m0_fid = 0.32
h_fid = 0.67
ns_fid = 0.96
sigma8_fid = 0.816
Omega_DE0_fid = 0.68
w0_fid = -1.0
wa_fid = 0.0
gamma_fid = 0.55

c = 299792.458
Aia = 1.72
Cia = 0.0134
nia = -0.41
bia = 2.17


In [4]:
L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))

len(L_array)

100

In [5]:
class Fisher:
    '''
    Calculate Fisher Matrix
    '''
    def __init__(self, params):
        self.num = params['num_params']
        self.universes = params['type']
        self.model = params['model']

    def trace(self, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia):
        F = np.zeros((self.num + 3, self.num + 3))
        L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))
        cosmic_params = {'l': L_array , 'z': np.linspace(0.001, 2.5, 50), 'type': self.universes, 'model': self.model}
        A = cs.CosmicShear(cosmic_params)
        
        f_sky = 0.3636
        epsilon = 0.01
        C = np.zeros((10, 10), dtype=object)
        dC_dq_Om_m_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_h_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_b_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_ns_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_sig_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Aia_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_nia_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_bia_matrix = np.zeros((10, 10), dtype=object)
        
        for i in range(10):
            for j in range(i, 10):
                logging.info(f"Calculated pair: {(i, j)}")
                C[i, j] = C[j, i] = A.Cosmic_Shear(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia)
                dC_dq_Om_m_matrix[i, j] = dC_dq_Om_m_matrix[j, i] = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia, 'Omega_m0')
                dC_dq_h_matrix[i, j] = dC_dq_h_matrix[j, i] = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia, 'h')
                dC_dq_Om_b_matrix[i, j] = dC_dq_Om_b_matrix[j, i] = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia, 'Omega_b0')
                dC_dq_ns_matrix[i, j] =  dC_dq_ns_matrix[j, i]  = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia, 'ns')
                dC_dq_sig_matrix[i, j] = dC_dq_sig_matrix[j, i] = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia, 'sigma8')
                dC_dq_Aia_matrix[i, j] =  dC_dq_Aia_matrix[j, i]  = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia, 'Aia')
                dC_dq_nia_matrix[i, j] =  dC_dq_nia_matrix[j, i]  = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia, 'nia')
                dC_dq_bia_matrix[i, j] =  dC_dq_bia_matrix[j, i]  = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, nia, bia, 'bia')
                
        C_matrix = np.array(C.tolist(), dtype=float)
        C_matrix_2d = np.squeeze(C_matrix)

        dC_dq_Om_m_matrix = np.array(dC_dq_Om_m_matrix.tolist(), dtype=float)
        dC_dq_h_matrix = np.array(dC_dq_h_matrix.tolist(), dtype=float)
        dC_dq_Om_b_matrix = np.array(dC_dq_Om_b_matrix.tolist(), dtype=float)
        dC_dq_ns_matrix = np.array(dC_dq_ns_matrix.tolist(), dtype=float)
        dC_dq_sig_matrix = np.array(dC_dq_sig_matrix.tolist(), dtype=float)
        dC_dq_Aia_matrix = np.array(dC_dq_Aia_matrix.tolist(), dtype=float)
        dC_dq_nia_matrix = np.array(dC_dq_nia_matrix.tolist(), dtype=float)
        dC_dq_bia_matrix = np.array(dC_dq_bia_matrix.tolist(), dtype=float)

        dC_dq_Om_m_matrix_2d = np.squeeze(dC_dq_Om_m_matrix)
        dC_dq_h_matrix_2d = np.squeeze(dC_dq_h_matrix)
        dC_dq_Om_b_matrix_2d = np.squeeze(dC_dq_Om_b_matrix)
        dC_dq_ns_matrix_2d = np.squeeze(dC_dq_ns_matrix)
        dC_dq_sig_matrix_2d = np.squeeze(dC_dq_sig_matrix)
        dC_dq_Aia_matrix_2d = np.squeeze(dC_dq_Aia_matrix)
        dC_dq_nia_matrix_2d = np.squeeze(dC_dq_nia_matrix)
        dC_dq_bia_matrix_2d = np.squeeze(dC_dq_bia_matrix)
        
        for i, l in enumerate(L_array):

            def lambda_k(i): 
                lambda_min = np.log10(10**L_array[0])
                lambda_max = np.log10(10**L_array[-1])
                delta_lambda = (lambda_max - lambda_min) / len(L_array)
                lambda_k = lambda_min + (i - 1)*delta_lambda
                return lambda_k
            
            delta_l = 10**lambda_k(i + 1) - 10**lambda_k(i)

            '''if i < len(L_array) - 1:
                delta_l = 10**L_array[i+1] - 10**L_array[i]
            else: 
                 delta_l = 1'''
                 
            L_parameter = np.sqrt((2 / ((2*(10**l) + 1)*delta_l*f_sky)))
            L_parameter_new = (2*(10 ** l) + 1) * delta_l / 2 * f_sky
            C_inv = np.linalg.inv(C_matrix_2d[:, :, i])

            #OMEGA_M 
            Om_m_Om_m = np.trace(C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_m_matrix_2d[:, :, i])
            Om_m_h = np.trace(C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i])
            Om_m_Om_b = np.trace(C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i])
            Om_m_ns = np.trace(C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i])
            Om_m_sig = np.trace(C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @  dC_dq_sig_matrix_2d[:, :, i])
            Om_m_Aia = np.trace(C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @  dC_dq_Aia_matrix_2d[:, :, i])
            Om_m_nia = np.trace(C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @  dC_dq_nia_matrix_2d[:, :, i])
            Om_m_bia = np.trace(C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @  dC_dq_bia_matrix_2d[:, :, i])
#

            #H
            h_h = np.trace(C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i])
            h_Om_b = np.trace(C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i])
            h_ns = np.trace(C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i])
            h_sig = np.trace(C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i])
            h_Aia = np.trace(C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i])
            h_nia = np.trace(C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_nia_matrix_2d[:, :, i])
            h_bia = np.trace(C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_bia_matrix_2d[:, :, i])
#
            #OMEGA_B
            Om_b_Om_b = np.trace(C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] )
            Om_b_ns = np.trace(C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] )
            Om_b_sig = np.trace(C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] )
            Om_b_Aia = np.trace(C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i] )
            Om_b_nia = np.trace(C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_nia_matrix_2d[:, :, i] )
            Om_b_bia = np.trace(C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_bia_matrix_2d[:, :, i] )

            #NS
            ns_ns = np.trace( C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] )
            ns_sig = np.trace( C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] )
            ns_Aia = np.trace( C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i] )
            ns_nia = np.trace( C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_nia_matrix_2d[:, :, i] )
            ns_bia = np.trace( C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_bia_matrix_2d[:, :, i] )

            #SIGMA8
            sig_sig = np.trace(C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] )
            sig_Aia = np.trace(C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i]  )
            sig_nia = np.trace(C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_nia_matrix_2d[:, :, i])
            sig_bia = np.trace(C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_bia_matrix_2d[:, :, i])

            #Aia
            Aia_Aia = np.trace(C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i]  )
            Aia_nia = np.trace(C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv @ dC_dq_nia_matrix_2d[:, :, i])
            Aia_bia = np.trace(C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv @ dC_dq_bia_matrix_2d[:, :, i])

            #nia
            nia_nia = np.trace(C_inv @ dC_dq_nia_matrix_2d[:, :, i] @ C_inv @ dC_dq_nia_matrix_2d[:, :, i])
            nia_bia = np.trace(C_inv @ dC_dq_nia_matrix_2d[:, :, i] @ C_inv @ dC_dq_bia_matrix_2d[:, :, i])

            #bia
            bia_bia = np.trace(C_inv @ dC_dq_bia_matrix_2d[:, :, i] @ C_inv @ dC_dq_bia_matrix_2d[:, :, i])



            if self.model == 'ACDM_flat':
                coeficientes = {
                        (0, 0): Om_m_Om_m, (0, 1): Om_m_h, (0, 2): Om_m_Om_b, (0, 3): Om_m_ns, (0, 4): Om_m_sig, (0, 5): Om_m_Aia, (0, 6): Om_m_nia, (0, 7): Om_m_bia,
                        (1, 1): h_h, (1, 2): h_Om_b, (1, 3): h_ns, (1, 4): h_sig, (1, 5): h_Aia, (1, 6): h_nia, (1, 7): h_bia,
                        (2, 2): Om_b_Om_b, (2, 3): Om_b_ns, (2, 4): Om_b_sig, (2, 5): Om_b_Aia, (2, 6): Om_b_nia, (2, 7): Om_b_bia,
                        (3, 3): ns_ns, (3, 4): ns_sig, (3, 5): ns_Aia, (3, 6): ns_nia, (3, 7): ns_bia,
                        (4, 4): sig_sig, (4, 5): sig_Aia, (4, 6): sig_nia, (4, 7): sig_bia,
                        (5, 5): Aia_Aia, (5, 6): Aia_nia, (5, 7): Aia_bia,
                        (6, 6): nia_nia, (6, 7): nia_bia,
                        (7, 7): bia_bia
                    }

                for (i, j), coef in coeficientes.items():
                    F[i, j] += (L_parameter_new * coef)

        Fisher = F + F.T - np.diag(F.diagonal())
        logging.info(f"Fisher Matrix: {Fisher}")

        return Fisher

In [6]:
params = {'num_params': 5, 'type': 'standard', 'model' : 'ACDM_flat'}

A = Fisher(params)

F = A.trace(Omega_m0_fid, h_fid, Omega_b0_fid, Omega_DE0_fid, w0_fid, wa_fid, ns_fid, sigma8_fid, gamma_fid, Aia, nia, bia)

Cov = np.linalg.inv(F)

print('Matriz de covarianzas')

print(' ')

print(Cov)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

Matriz de covarianzas
 
[[ 3.08811780e-06  1.29653529e-05  1.84384265e-06 -6.26130064e-06
   2.42729306e-05 -2.03811916e-04  7.96785717e-06  1.24863085e-06]
 [ 1.29653529e-05  5.71697750e-05  7.80900738e-06 -2.78403069e-05
   9.86463562e-05 -8.60051841e-04  3.63396191e-05  5.29955584e-06]
 [ 1.84384265e-06  7.80900738e-06  1.11258772e-06 -3.73387245e-06
   1.45916474e-05 -1.22996788e-04  5.15256615e-06  6.74543898e-07]
 [-6.26130064e-06 -2.78403069e-05 -3.73387245e-06  1.53185669e-05
  -4.63444193e-05  4.01929168e-04 -1.17802002e-05 -2.92528761e-06]
 [ 2.42729306e-05  9.86463562e-05  1.45916474e-05 -4.63444193e-05
   1.98818581e-04 -1.62268309e-03  6.50945275e-05  8.50090901e-06]
 [-2.03811916e-04 -8.60051841e-04 -1.22996788e-04  4.01929168e-04
  -1.62268309e-03  1.37178476e-02 -6.41975553e-04 -6.28178086e-05]
 [ 7.96785717e-06  3.63396191e-05  5.15256615e-06 -1.17802002e-05
   6.50945275e-05 -6.41975553e-04  9.04460022e-05 -8.65188589e-06]
 [ 1.24863085e-06  5.29955584e-06  6.74543898

In [7]:
def comparison(created, expected):
    return 100*np.abs(1 - (created/expected))

In [8]:
print('ERRORES')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error ns: ' + str(np.sqrt(Cov[3, 3])))
print('Error sigma8: ' + str(np.sqrt(Cov[4, 4])))
print('Error Aia: ' + str(np.sqrt(Cov[5, 5])))
print('Error nia: ' + str(np.sqrt(Cov[6, 6])))
print('Error bia: ' + str(np.sqrt(Cov[7, 7])))

print(' ')

# Relative errors

print('RELATIVE ERRORS')

err_Om_m = (np.sqrt(Cov[0, 0]) / Omega_m0_fid)
err_h = np.sqrt(Cov[1, 1]) / h_fid
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0_fid
err_ns = np.sqrt(Cov[3, 3]) / ns_fid
err_sig = np.sqrt(Cov[4, 4]) / sigma8_fid
err_A = np.sqrt(Cov[5, 5]) / Aia
err_nia = np.sqrt(Cov[6, 6]) / Cia
err_bia = np.sqrt(Cov[7, 7]) / bia

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))
print('Error Relativo A: ' + str(err_A))
print('Error Relativo nia: ' + str(err_nia))
print('Error Relativo bia: ' + str(err_bia))

print(' ')

print('COMPARACION')
## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.018
S_h = 0.21
S_Om_b = 0.47
S_ns = 0.035
S_sig = 0.0087

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

ERRORES
Error Omega_m: 0.001757304127334119
Error h: 0.007561069701861793
Error Omega_b: 0.0010547927381760888
Error ns: 0.003913894085403967
Error sigma8: 0.014100304286172384
Error Aia: 0.11712321531755493
Error nia: 0.009510310308676585
Error bia: 0.0017719122416153539
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.0054915753979191215
Error Relativo h: 0.011285178659495212
Error Relativo Omega_b: 0.021095854763521774
Error Relativo ns: 0.004076973005629133
Error Relativo sigma8: 0.017279784664426943
Error Relativo A: 0.06809489262648542
Error Relativo nia: 0.7097246499012377
Error Relativo bia: 0.0008165494200992415
 
COMPARACION
Comparison for Omega_m: 69.49124778933822
Comparison for h: 94.62610540024038
Comparison for Omega_b: 95.51152026308047
Comparison for ns: 88.35150569820249
Comparison for sigma8: 98.61821453364303


In [9]:
import numpy as np
print(0.0055 / 0.816)

0.006740196078431373


**Multiplicando por s8^2**

COMPARACION
Comparison for Omega_m: 110.03216424270565
Comparison for h: 52.375206993023006
Comparison for Omega_b: 61.8563146904604
Comparison for ns: 33.59939198274014
Comparison for sigma8: 659.7156450844985

**P normal**

COMPARACION
Comparison for Omega_m: 64.11380658925354
Comparison for h: 61.87769435839172
Comparison for Omega_b: 69.18282269637388
Comparison for ns: 46.73304944792112
Comparison for sigma8: 503.5311383558334

**Cambiando l a log pero con P normal**

COMPARACION
Comparison for Omega_m: 97.53474353468317
Comparison for h: 99.18926490645329
Comparison for Omega_b: 99.33859228237915
Comparison for ns: 99.12369894128061
Comparison for sigma8: 88.1360267046474

**l con log10 y h = 1**

COMPARACION
Comparison for Omega_m: 120.02346863938777
Comparison for h: 51.820746461561704
Comparison for Omega_b: 64.2780590048458
Comparison for ns: 31.751269686394846
Comparison for sigma8: 671.5045554646256

**l con log10, h = 1, dC/ds8 = 2 C / s8**

RELATIVE ERRORS
Error Relativo Omega_m: 0.025569499122491096
Error Relativo h: 0.02544447397976539
Error Relativo Omega_b: 0.12781853726688738
Error Relativo ns: 0.019258716654957552
Error Relativo sigma8: 0.000283201928209537
Error Relativo A: 4.0985175607906195
Error Relativo nia: 335.1993020878382
Error Relativo bia: 0.8803604387468326

**P_s8_pl / h**3**

RELATIVE ERRORS
Error Relativo Omega_m: 0.024357011973011967
Error Relativo h: 0.059902893359825876
Error Relativo Omega_b: 0.04475554971089151
Error Relativo ns: 0.01887256724779941
Error Relativo sigma8: 0.000282367230474732
Error Relativo A: 4.094057426807773
Error Relativo nia: 334.83721619145575
Error Relativo bia: 0.880045333324018

**P * s8^2**

COMPARACION
Comparison for Omega_m: 81.69201327658915
Comparison for h: 84.59259691521453
Comparison for Omega_b: 68.51592090495913
Comparison for ns: 33.639764151061314
Comparison for sigma8: 96.7658189103658

**Misma implementacion de k que con Fisher para los C**

COMPARACION
Comparison for Omega_m: 63.72193053322817
Comparison for h: 86.13414038452535
Comparison for Omega_b: 71.29516317710069
Comparison for ns: 37.74840016472957
Comparison for sigma8: 96.73498728819303

**Lo mismo de arriba con h =1**

COMPARACION
Comparison for Omega_m: 28.292847713244274
Comparison for h: 89.07787293496634
Comparison for Omega_b: 75.27503680207079
Comparison for ns: 48.00673614841256
Comparison for sigma8: 96.7128321837372